In [87]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time
from __future__ import print_function
from builtins import input


from sporco.admm import bpdn
### using the ADMM algorithm 
### we can also use the PGM algorithm 
from sporco import util
from sporco import plot
plot.config_notebook_plotting()

from scipy.linalg import hadamard

import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

import time 
from scipy import stats
from matplotlib import pyplot
#plt.style.use('ggplot')
plt.style.use('seaborn-dark-palette')
#iHiV = pd.read_pickle("~/bge-analysis-simv3/iHiV.pkl")
#B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")


import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_absolute_percentage_error as mape



import matplotlib.font_manager as fm


fontSize=12
fontSizeSmall=10
labelSize=8


machine_precision=1e-6



SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 14

BIGGEST_SIZE = 18
# plt.rc('font', size=SMALL_SIZE, family='sans-serif', serif='Arial')          # controls default text sizes
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE, titleweight='bold')     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE, labelweight='bold')    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('text')


gs_font = fm.FontProperties(fname='gillsans.ttf', size=20, weight='bold')
light_grey=(0.5,0.5,0.5)
from sklearn.metrics import r2_score

import seaborn as sns

import scipy.stats as sct

In [88]:
labdl = np.round(np.logspace(-5,0.8,5),5)
lam = list(labdl)
lam.append([7.0,9.0,10.0,15.0,20.0])
lamf = list(pd.DataFrame(lam)[0].explode().values)
lamf.sort()

In [44]:
sst = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/16-list1-allmetabols.pkl")

In [90]:
errdf =  pd.read_pickle("7n10_cv_WHmetabols.pkl")
errdf1 = pd.read_pickle("3n5_cv_WHsuccinate.pkl")
errdf2 = pd.read_pickle("3n5_cv_WHmetabols.pkl")
cv_splitsl = [7,10]
errlindf = pd.read_pickle("7n10_cv_linmetabols.pkl")
errlindf1 = pd.read_pickle("3n5_cv_l1succinate.pkl")
errlindf2 = pd.read_pickle("3n5_cv_linmetabols.pkl")

In [91]:
errdff = pd.concat([errdf,errdf1,errdf2])
errlindff = pd.concat([errlindf,errlindf1,errlindf2])

#errdff = errdf.copy()
#errlindff = errlindf.copy()

In [92]:
metabolite_list = ["butyrate","acetate","lactate","succinate"]

In [93]:
laml = errdff[1].unique()

In [94]:
buty = errdff[errdff[2]=="butyrate"]

In [95]:
crossval = [] 
for m in [3,5,7,10]:
    samp = errdff[errdff[7]==m]
    for runs in range(3):
        rundf = samp[samp[0]==runs]
        for metabols in ["butyrate","acetate","lactate","succinate"]:
            sstm = sst[str(metabols)]
            sp1 = rundf[rundf[2]==str(metabols)]
            laml = sp1[1].unique()
            for rhola in laml:
                rho1 = sp1[sp1[1]==rhola]
                r2 = [] 
                mape2 = [] 
                yallbest  = []
                yallact  = [] 
                for iterii in range(m):
                    #idx = rho1[3].iloc[iterii]
                    yact = rho1[4].iloc[iterii].ravel()
                    ybest = rho1[3].iloc[iterii]
                    ybestdf1 = pd.DataFrame(ybest)
                    ybestdf1[ybestdf1[0]<1e-10]=0
                    yallbest.append(ybestdf1[0].values)
                    yallact.append(yact)
                    slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                    rval = r2_score(yact,ybestdf1[0].values)
                    gop = 1/(2-rval)
                    mape1 = mape(yact,ybestdf1[0].values)
                    r2.append([gop,rval,mape1,slope,intercept,r_value**2])
                r2df = pd.DataFrame(r2)
                if m == 2:
                    ybestagg = [*yallbest[0],*yallbest[1]]
                    yactagg = [*yallact[0],*yallact[1]]
                if m == 3:
                    ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
                    yactagg = [*yallact[0],*yallact[1],*yallact[2]]
                if m == 5:
                    ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
                    yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
                if m == 7:
                    ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
                    yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
                if m == 9:
                    ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8]]
                    yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8]]
                if m == 10:
                    ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8],*yallbest[9]]
                    yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8],*yallact[9]]
                slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
                rval2 = r2_score(yactagg,ybestagg)
                gop2 = 1/(2-rval2)
                mean1 = r2df.mean().values[0]
                mean2 = r2df.mean().values[1]
                mean3 = r2df.mean().values[2]
                mean4 = r2df.mean().values[3]
                mean5 = r2df.mean().values[4]
                mean6 = r2df.mean().values[5]
                mape2 = mape(yactagg,ybestagg)
                #print(metabols)
                crossval.append([runs,m,str(metabols),rhola,gop2,rval2,mape2,slope,intercept,r_value**2,"break",mean1,mean2,mean3,mean4,mean5,mean6,yactagg,ybestagg])

In [96]:
crossvalin = [] 
for m in [3,5,7,10]:
    samp = errlindff[errlindff[7]==m]
    for runs in range(3):
        rundf = samp[samp[0]==runs]
        for metabols in ["butyrate","acetate","lactate","succinate"]:
            sstm = sst[str(metabols)]
            sp1 = rundf[rundf[2]==str(metabols)]
            laml = sp1[1].unique()
            for rhola in laml:
                rho1 = sp1[sp1[1]==rhola]
                r2 = [] 
                mape2 = [] 
                yallbest  = []
                yallact  = [] 
                for iterii in range(m):
                    #idx = rho1[3].iloc[iterii]
                    yact = rho1[4].iloc[iterii].ravel()
                    ybest = rho1[3].iloc[iterii]
                    ybestdf1 = pd.DataFrame(ybest)
                    ybestdf1[ybestdf1[0]<1e-10]=0
                    yallbest.append(ybestdf1[0].values)
                    yallact.append(yact)
                    slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                    rval = r2_score(yact,ybestdf1[0].values)
                    gop = 1/(2-rval)
                    mape1 = mape(yact,ybestdf1[0].values)
                    r2.append([gop,rval,mape1,slope,intercept,r_value**2])
                r2df = pd.DataFrame(r2)
                if m == 2:
                    ybestagg = [*yallbest[0],*yallbest[1]]
                    yactagg = [*yallact[0],*yallact[1]]
                if m == 3:
                    ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
                    yactagg = [*yallact[0],*yallact[1],*yallact[2]]
                if m == 5:
                    ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
                    yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
                if m == 7:
                    ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
                    yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
                if m == 9:
                    ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8]]
                    yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8]]
                if m == 10:
                    ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8],*yallbest[9]]
                    yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8],*yallact[9]]
                slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
                rval2 = r2_score(yactagg,ybestagg)
                gop2 = 1/(2-rval2)
                mean1 = r2df.mean().values[0]
                mean2 = r2df.mean().values[1]
                mean3 = r2df.mean().values[2]
                mean4 = r2df.mean().values[3]
                mean5 = r2df.mean().values[4]
                mean6 = r2df.mean().values[5]
                mape2 = mape(yactagg,ybestagg)
                #print(metabols)
                crossvalin.append([runs,m,str(metabols),rhola,gop2,rval2,mape2,slope,intercept,r_value**2,"break",mean1,mean2,mean3,mean4,mean5,mean6,yactagg,ybestagg])

In [97]:
crossdf = pd.DataFrame(crossval,columns=["run","k-fold","sp","lambda","nnseagg","r2agg","mapeagg","slagg","intagg","rlinagg","means on right","nnseavg","r2avg","mapeavg","slavg","intavg","rlinavg","testact","testpred"])

In [98]:
crosslindf = pd.DataFrame(crossvalin,columns=["run","k-fold","sp","lambda","nnseagg","r2agg","mapeagg","slagg","intagg","rlinagg","means on right","nnseavg","r2avg","mapeavg","slavg","intavg","rlinavg","testact","testpred"])

In [99]:
crossdf.head(2)

,run,k-fold,sp,lambda,nnseagg,r2agg,mapeagg,slagg,intagg,rlinagg,means on right,nnseavg,r2avg,mapeavg,slavg,intavg,rlinavg,testact,testpred
0,0,3,butyrate,0.0000,0.794895,0.741973,1.765799e+15,0.761481,4.082771,0.742943,break,0.792573,0.735016,1.768871e+15,0.763859,3.955216,0.744537,"[46.5489, 41.2675916, 13.037733333333335, 25.9...","[43.66105670716413, 45.98674320963104, 8.32469..."
1,0,3,butyrate,0.0003,0.730041,0.630213,1.160639e+15,0.787351,5.006277,0.660118,break,0.729961,0.617418,1.161974e+15,0.793559,4.870349,0.670334,"[46.5489, 41.2675916, 13.037733333333335, 25.9...","[53.987314981821854, 52.44565389656837, 11.416..."


In [100]:
crosslindf.head(2)

,run,k-fold,sp,lambda,nnseagg,r2agg,mapeagg,slagg,intagg,rlinagg,means on right,nnseavg,r2avg,mapeavg,slavg,intavg,rlinavg,testact,testpred
0,0,3,butyrate,0.0000,0.707996,0.587563,1.529892e+15,0.896079,3.395819,0.672247,break,0.708222,0.572614,1.532872e+15,0.910389,3.087718,0.692175,"[46.5489, 41.2675916, 13.037733333333335, 25.9...","[52.31209207735067, 49.7609735371106, 8.989004..."
1,0,3,butyrate,0.0003,0.689281,0.549212,1.227720e+15,0.902181,3.486351,0.655340,break,0.690425,0.532862,1.226152e+15,0.908832,3.336752,0.665191,"[46.5489, 41.2675916, 13.037733333333335, 25.9...","[51.838486675876, 56.30587014891403, 12.040246..."


In [101]:
pd.to_pickle(crossdf,"WH-REL-analysis-metabALLcv.pkl")
pd.to_pickle(crosslindf,"sigma-REL-analysis-metabALLcv.pkl")

In [102]:
crossdf.head(3)

,run,k-fold,sp,lambda,nnseagg,r2agg,mapeagg,slagg,intagg,rlinagg,means on right,nnseavg,r2avg,mapeavg,slavg,intavg,rlinavg,testact,testpred
0,0,3,butyrate,0.0000,0.794895,0.741973,1.765799e+15,0.761481,4.082771,0.742943,break,0.792573,0.735016,1.768871e+15,0.763859,3.955216,0.744537,"[46.5489, 41.2675916, 13.037733333333335, 25.9...","[43.66105670716413, 45.98674320963104, 8.32469..."
1,0,3,butyrate,0.0003,0.730041,0.630213,1.160639e+15,0.787351,5.006277,0.660118,break,0.729961,0.617418,1.161974e+15,0.793559,4.870349,0.670334,"[46.5489, 41.2675916, 13.037733333333335, 25.9...","[53.987314981821854, 52.44565389656837, 11.416..."
2,0,3,butyrate,0.0079,0.730781,0.631600,1.147865e+15,0.789113,4.961729,0.661487,break,0.730638,0.618886,1.149364e+15,0.795479,4.823088,0.671751,"[46.5489, 41.2675916, 13.037733333333335, 25.9...","[53.84725843544344, 52.50767509447481, 11.5446..."


In [202]:
whcv = []
for metabols in metabolite_list :
    whdf = crossdf[crossdf["sp"]==metabols]
    for cv in [3,5,7,10]:
        cvdf2 = whdf[whdf["k-fold"]==cv]
        for runs in range(3):
            run2 = cvdf2[cvdf2["run"]==runs]
            lit = run2.T[run2["nnseagg"].idxmax()]
            whcv.append([metabols,cv,runs,lit["lambda"],lit["nnseagg"]])

whdf = pd.DataFrame(whcv)
avgwh = [] 
for mb in metabolite_list :
    mbdf2 = whdf[whdf[0]==mb]
    for cv in [3,5,7,10]:
        cvdf3 = mbdf2[mbdf2[1]==cv]
        l1 = sct.mode(cvdf3[3])[0][0]
        l2 = sct.mode(cvdf3[3])[1][0]
        
        avgwh.append([cv,mb,cvdf3[4].mean(),cvdf3[4].std(),l1,l2])

meanWH = pd.DataFrame(avgwh)
meanWH.to_pickle("WH_OV_functions.pkl")

In [203]:
whall = whdf.copy()

In [204]:
whcv = []
for metabols in metabolite_list :
    whdf = crosslindf[crosslindf["sp"]==metabols]
    for cv in [3,5,7,10]:
        cvdf2 = whdf[whdf["k-fold"]==cv]
        for runs in range(3):
            run2 = cvdf2[cvdf2["run"]==runs]
            lit = run2.T[run2["nnseagg"].idxmax()]
            whcv.append([metabols,cv,runs,lit["lambda"],lit["nnseagg"]])

whdf = pd.DataFrame(whcv)
avgwh = [] 
for mb in metabolite_list :
    mbdf2 = whdf[whdf[0]==mb]
    for cv in [3,5,7,10]:
        cvdf3 = mbdf2[mbdf2[1]==cv]
        l1 = sct.mode(cvdf3[3])[0][0]
        l2 = sct.mode(cvdf3[3])[1][0]
        
        avgwh.append([cv,mb,cvdf3[4].mean(),cvdf3[4].std(),l1,l2])

meansig = pd.DataFrame(avgwh)
meansig.to_pickle("l1_OV_functions.pkl")

In [205]:
l1all = whdf.copy()

In [106]:
rdf = pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-16sp-FUNCTIONS-v3/RF-OV-functions.pkl")

In [107]:
rfa = [] 

cv_splitsl = [3,5,7] 
max_depthl = [2,5,10,15,20]
ntrees = [100]
#max_featuresl = ["auto","sqrt","log2"]
max_featuresl = [0.33]

for metabols in rdf["metabols"].unique():
    mdf = rdf[rdf["metabols"]==str(metabols)]
    for runs in range(3):
        rundf = mdf[mdf["runs"]==runs]
        for m in [3,5,7,10]:
            for md in max_depthl:
                rho1 = rundf[rundf["md"]==md]
                for nest in ntrees: 
                    ntre = rho1[rho1["trees"]==nest]
                    for mfeats in max_featuresl:
                        m2df = ntre[ntre["feats"]==mfeats]
                        r2 = [] 
                        mape2 = [] 
                        yallbest  = []
                        yallact  = [] 
                        for iterii in range(m):
                            yact  = m2df["yact"].iloc[iterii]
                            ybest = m2df["ypred"].iloc[iterii]
                            ybestdf1 = pd.DataFrame(ybest)
                            ybestdf1[ybestdf1[0]<1e-10]=0
                            slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                            rval = r2_score(yact,ybestdf1[0].values)
                            gop = 1/(2-rval)
                            r2.append([gop,r2_score(yact,ybestdf1[0].values)])
                            mape2.append(mape(yact,ybestdf1[0].values))
                            yallbest.append(ybestdf1[0].values)
                            yallact.append(yact)
                            slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                            rval = r2_score(yact,ybestdf1[0].values)
                            gop = 1/(2-rval)
                            mape1 = mape(yact,ybestdf1[0].values)
                            r2.append([gop,rval,mape1,slope,intercept,r_value**2])
                        r2df = pd.DataFrame(r2)
                        if m == 2:
                            ybestagg = [*yallbest[0],*yallbest[1]]
                            yactagg = [*yallact[0],*yallact[1]]
                        if m == 3:
                            ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
                            yactagg = [*yallact[0],*yallact[1],*yallact[2]]
                        if m == 5:
                            ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
                            yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
                        if m == 6:
                            ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5]]
                            yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5]]
                        if m == 7:
                            ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
                            yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
                        if m == 9:
                            ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8]]
                            yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8]]
                        if m == 10:
                            ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8],*yallbest[9]]
                            yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8],*yallact[9]]
                        slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
                        rval2 = r2_score(yactagg,ybestagg)
                        gop2 = 1/(2-rval2)
                        mean1 = r2df.mean().values[0]
                        mean2 = r2df.mean().values[1]
                        mean3 = r2df.mean().values[2]
                        mean4 = r2df.mean().values[3]
                        mean5 = r2df.mean().values[4]
                        mean6 = r2df.mean().values[5]
                        mape2 = mape(yactagg,ybestagg)
                        rfa.append([runs,m,metabols,md,nest,mfeats,gop2,rval2,mape2,slope,intercept,r_value**2,"break",mean1,mean2,mean3,mean4,mean5,mean6,yactagg,ybestagg])
                            

In [108]:
crfadf = pd.DataFrame(rfa,columns=["runidx","k-fold","sp","max_depth","trees","features","nnseagg","r2agg","mapeagg","slagg","intagg","rlinagg","means on right","nnseavg","r2avg","mapeavg","slavg","intavg","rlinavg","testact","testpred"])

In [110]:
bestRF = []
for metabols in metabolite_list:
    pdf  = crfadf[crfadf["sp"]==metabols]
    for runs in range(3):
        rundf1 = pdf[pdf["runidx"]==runs]
        for cv in [3,5,7,10]:
            cvdf = rundf1[rundf1["k-fold"]==cv]
            lit = cvdf.T[cvdf["nnseagg"].idxmax()]
            bestRF.append([metabols,cv,runs,lit["max_depth"],lit["nnseagg"],lit["r2agg"]])

In [194]:
allrf = brdf[brdf[1]==10]

In [206]:
allwh = whall[whall[1]==10]

In [207]:
all1 = l1all[l1all[1]==10]

In [111]:
brdf  = pd.DataFrame(bestRF)

In [137]:
bestavg  = []
for cv in [3,5,7,10]:
    cvdf = brdf[brdf[1]==cv]
    for metabols in metabolite_list:
        mpdf = cvdf[cvdf[0]==metabols]
        l1 = sct.mode(mpdf[3])[0][0]
        l2 = sct.mode(mpdf[3])[1][0]
        bestavg.append([cv,metabols,mpdf[4].mean(),mpdf[4].std(),l1,l2])
    

In [139]:
bestavg

[[3, 'butyrate', 0.8276223448722088, 0.005564832634193929, 15, 3],
 [3, 'acetate', 0.541599401291181, 0.013042503010835529, 5, 2],
 [3, 'lactate', 0.7487025902414812, 0.006099272425875082, 20, 3],
 [3, 'succinate', 0.8134056291313434, 0.015948066267685034, 10, 2],
 [5, 'butyrate', 0.8270236484410834, 0.005267208805265422, 15, 3],
 [5, 'acetate', 0.5418449138491049, 0.012762448750763319, 5, 2],
 [5, 'lactate', 0.7449957090652936, 0.0033656277895582737, 20, 2],
 [5, 'succinate', 0.818762457723896, 0.011540654844417898, 10, 2],
 [7, 'butyrate', 0.8304885758839694, 0.004371603051402273, 15, 2],
 [7, 'acetate', 0.5438272274667052, 0.007848365432624429, 5, 2],
 [7, 'lactate', 0.748190492125711, 0.0023346226475677956, 10, 1],
 [7, 'succinate', 0.815924386876668, 0.012428970181893503, 10, 2],
 [10, 'butyrate', 0.8313274507951047, 0.004411771118937098, 15, 2],
 [10, 'acetate', 0.5459460052975689, 0.008934244202030952, 5, 3],
 [10, 'lactate', 0.7527633053270965, 0.004913999859632091, 10, 2],
 [1

In [140]:
avgdf = pd.DataFrame(bestavg)

In [141]:
avgdf.to_pickle("RF-OVfunctionsavg.pkl")

In [143]:
meanWH[meanWH[0]==3]

,0,1,2,3,4,5,method
0,3,butyrate,0.821309,0.006296,9.00000,2,WH
4,3,acetate,0.550758,0.024978,20.00000,3,WH
8,3,lactate,0.743156,0.025162,7.00000,1,WH
12,3,succinate,0.825850,0.011867,6.30957,3,WH


In [144]:
meansig[meansig[0]==3]

,0,1,2,3,4,5,method
0,3,butyrate,0.808615,0.004165,10.00000,2,L1
4,3,acetate,0.525601,0.014970,20.00000,3,L1
8,3,lactate,0.714270,0.021211,9.00000,1,L1
12,3,succinate,0.782766,0.032199,6.30957,3,L1


In [145]:
avgdf[avgdf[0]==3]

,0,1,2,3,4,5
0,3,butyrate,0.827622,0.005565,15,3
1,3,acetate,0.541599,0.013043,5,2
2,3,lactate,0.748703,0.006099,20,3
3,3,succinate,0.813406,0.015948,10,2


In [146]:
meanWH["method"] = ["WH"]*len(meanWH)
meansig["method"] = ["L1"]*len(meansig)
avgdf["method"] = ["RF"]*len(avgdf)

In [147]:
meansig.head(2)

,0,1,2,3,4,5,method
0,3,butyrate,0.808615,0.004165,10.0000,2,L1
1,5,butyrate,0.825408,0.005108,6.3096,1,L1


In [149]:
avgdf.head(4)

,0,1,2,3,4,5,method
0,3,butyrate,0.827622,0.005565,15,3,RF
1,3,acetate,0.541599,0.013043,5,2,RF
2,3,lactate,0.748703,0.006099,20,3,RF
3,3,succinate,0.813406,0.015948,10,2,RF


In [151]:
fdf = pd.concat([meanWH,meansig,avgdf])

In [153]:
fdf.to_pickle("OV_funcs_allCV_allmethods.pkl")

In [185]:
comprf = avgdf[avgdf[0]==10]
avgdf

,0,1,2,3,4,5,method
0,3,butyrate,0.827622,0.005565,15,3,RF
1,3,acetate,0.541599,0.013043,5,2,RF
2,3,lactate,0.748703,0.006099,20,3,RF
3,3,succinate,0.813406,0.015948,10,2,RF
4,5,butyrate,0.827024,0.005267,15,3,RF
5,5,acetate,0.541845,0.012762,5,2,RF
6,5,lactate,0.744996,0.003366,20,2,RF
7,5,succinate,0.818762,0.011541,10,2,RF
8,7,butyrate,0.830489,0.004372,15,2,RF
9,7,acetate,0.543827,0.007848,5,2,RF


In [217]:
compwh = meanWH[meanWH[0]==7]
compl1 = meansig[meansig[0]==7]
all1

,0,1,2,3,4
9,butyrate,10,0,7.00000,0.825739
10,butyrate,10,1,6.30957,0.838279
11,butyrate,10,2,12.00000,0.832524
21,acetate,10,0,20.00000,0.545964
22,acetate,10,1,20.00000,0.535515
23,acetate,10,2,20.00000,0.568647
33,lactate,10,0,7.00000,0.760791
34,lactate,10,1,7.00000,0.762625
35,lactate,10,2,9.00000,0.755930
45,succinate,10,0,7.00000,0.802318


In [224]:
print("means=" ,allwh[4].mean(),all1[4].mean())
#h.index = np.arange(0,5,1)

def statistic(x, y, axis):

    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

statistic(compwh[2],comprf[2],0)
from scipy import stats 
res = stats.permutation_test((allwh[4],all1[4]),statistic, vectorized=True,

                       n_resamples=8000, alternative='greater',permutation_type='samples')

print("statistic= ",res.statistic)

print("p-value = ",res.pvalue)
#res.null_distribution

def stars(p):
   if p < 0.0001:
       return "****"
   elif (p < 0.001):
       return "***"
   elif (p < 0.01):
       return "**"
   elif (p < 0.05):
       return "*"
   else:
       return "ns"    

means= 0.755014078773978 0.7362753015092832
statistic=  0.018738777264694972
p-value =  0.0


In [189]:
statistic(compwh[2],comprf[2],0)

0.01599034615443551

In [209]:
all1

,0,1,2,3,4
9,butyrate,10,0,7.00000,0.825739
10,butyrate,10,1,6.30957,0.838279
11,butyrate,10,2,12.00000,0.832524
21,acetate,10,0,20.00000,0.545964
22,acetate,10,1,20.00000,0.535515
23,acetate,10,2,20.00000,0.568647
33,lactate,10,0,7.00000,0.760791
34,lactate,10,1,7.00000,0.762625
35,lactate,10,2,9.00000,0.755930
45,succinate,10,0,7.00000,0.802318


In [210]:
allwh

,0,1,2,3,4
9,butyrate,10,0,9.00000,0.844694
10,butyrate,10,1,6.30957,0.852770
11,butyrate,10,2,6.30957,0.853751
21,acetate,10,0,20.00000,0.562906
22,acetate,10,1,20.00000,0.535984
23,acetate,10,2,20.00000,0.584401
33,lactate,10,0,7.00000,0.780586
34,lactate,10,1,9.00000,0.789756
35,lactate,10,2,10.00000,0.781985
45,succinate,10,0,6.30957,0.830217
